<a href="https://colab.research.google.com/github/mohameddhameem/IBM-Machine-Learning/blob/master/Deep%20Learning%20and%20Reinforcement%20Learning/Week%204/05g_DEMO_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning Foundation

## Course 5, Part g: RNN DEMO

## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).

In [2]:
from tensorflow import keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.datasets import imdb
from tensorflow.keras import initializers

In [3]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [4]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

17473536/17464789 [==============================] - 0s 0us/step
25000 train sequences
25000 test sequences


In [5]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [6]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99], dtype=int32)

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [7]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [8]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          1000000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 5)                 280       
                                                                 
 dense (Dense)               (None, 1)                 6         
                                                                 
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [9]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [10]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 30s 33ms/step - loss: 0.6725 - accuracy: 0.5510 - val_loss: 0.6425 - val_accuracy: 0.6228
Epoch 2/10
782/782 [==============================] - 25s 33ms/step - loss: 0.6186 - accuracy: 0.6986 - val_loss: 0.6133 - val_accuracy: 0.7366
Epoch 3/10
782/782 [==============================] - 25s 32ms/step - loss: 0.5900 - accuracy: 0.7448 - val_loss: 0.5964 - val_accuracy: 0.7214
Epoch 4/10
782/782 [==============================] - 25s 32ms/step - loss: 0.5686 - accuracy: 0.7669 - val_loss: 0.5825 - val_accuracy: 0.7350
Epoch 5/10
782/782 [==============================] - 25s 33ms/step - loss: 0.5496 - accuracy: 0.7796 - val_loss: 0.5699 - val_accuracy: 0.7404
Epoch 6/10
782/782 [==============================] - 25s 32ms/step - loss: 0.5332 - accuracy: 0.7902 - val_loss: 0.5545 - val_accuracy: 0.7556
Epoch 7/10
782/782 [==============================] - 26s 33ms/step - loss: 0.5186 - accuracy: 0.7982 - val_loss: 0.5487 - val_accuracy:

In [11]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 3s 4ms/step - loss: 0.5188 - accuracy: 0.7741
Test score: 0.5188170075416565
Test accuracy: 0.7740799784660339


## Exercise

In this exercise, we will illustrate:
- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?

In [12]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [13]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [14]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [15]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 63s 79ms/step - loss: 0.5989 - accuracy: 0.6669 - val_loss: 0.5112 - val_accuracy: 0.7454
Epoch 2/10
782/782 [==============================] - 61s 78ms/step - loss: 0.4418 - accuracy: 0.7961 - val_loss: 0.4446 - val_accuracy: 0.7900
Epoch 3/10
782/782 [==============================] - 61s 78ms/step - loss: 0.3731 - accuracy: 0.8374 - val_loss: 0.3992 - val_accuracy: 0.8198
Epoch 4/10
782/782 [==============================] - 62s 79ms/step - loss: 0.3322 - accuracy: 0.8595 - val_loss: 0.3921 - val_accuracy: 0.8242
Epoch 5/10
782/782 [==============================] - 61s 78ms/step - loss: 0.3056 - accuracy: 0.8734 - val_loss: 0.3720 - val_accuracy: 0.8327
Epoch 6/10
782/782 [==============================] - 66s 84ms/step - loss: 0.2868 - accuracy: 0.8808 - val_loss: 0.3648 - val_accuracy: 0.8385
Epoch 7/10
782/782 [==============================] - 60s 77ms/step - loss: 0.2729 - accuracy: 0.8870 - val_loss: 0.3704 - val_accuracy:

In [16]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [17]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [18]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [19]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 65s 83ms/step - loss: 0.6470 - accuracy: 0.6217 - val_loss: 0.5749 - val_accuracy: 0.7127
Epoch 2/10
782/782 [==============================] - 64s 82ms/step - loss: 0.5245 - accuracy: 0.7442 - val_loss: 0.5209 - val_accuracy: 0.7418
Epoch 3/10
782/782 [==============================] - 64s 82ms/step - loss: 0.4714 - accuracy: 0.7777 - val_loss: 0.4864 - val_accuracy: 0.7606
Epoch 4/10
782/782 [==============================] - 65s 83ms/step - loss: 0.4373 - accuracy: 0.7990 - val_loss: 0.4660 - val_accuracy: 0.7762
Epoch 5/10
782/782 [==============================] - 65s 83ms/step - loss: 0.4084 - accuracy: 0.8174 - val_loss: 0.4388 - val_accuracy: 0.7956
Epoch 6/10
782/782 [==============================] - 65s 83ms/step - loss: 0.3833 - accuracy: 0.8333 - val_loss: 0.4499 - val_accuracy: 0.7939
Epoch 7/10
782/782 [==============================] - 65s 83ms/step - loss: 0.3653 - accuracy: 0.8412 - val_loss: 0.4144 - val_accuracy:

In [20]:
# Out of curiosity, run for 10 more epochs
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 65s 83ms/step - loss: 0.3257 - accuracy: 0.8618 - val_loss: 0.3849 - val_accuracy: 0.8269
Epoch 2/10
782/782 [==============================] - 65s 83ms/step - loss: 0.3185 - accuracy: 0.8646 - val_loss: 0.3802 - val_accuracy: 0.8295
Epoch 3/10
782/782 [==============================] - 65s 83ms/step - loss: 0.3126 - accuracy: 0.8673 - val_loss: 0.3753 - val_accuracy: 0.8310
Epoch 4/10
782/782 [==============================] - 65s 83ms/step - loss: 0.3076 - accuracy: 0.8708 - val_loss: 0.3724 - val_accuracy: 0.8315
Epoch 5/10
782/782 [==============================] - 65s 83ms/step - loss: 0.3024 - accuracy: 0.8724 - val_loss: 0.3724 - val_accuracy: 0.8321
Epoch 6/10
782/782 [==============================] - 69s 89ms/step - loss: 0.2990 - accuracy: 0.8744 - val_loss: 0.3752 - val_accuracy: 0.8318
Epoch 7/10
782/782 [==============================] - 65s 83ms/step - loss: 0.2952 - accuracy: 0.8764 - val_loss: 0.3718 - val_accuracy:

---
### Machine Learning Foundation (C) 2020 IBM Corporation